### Load the CSV Data and Fix Paths

In [ ]:
import cv2
import csv
import numpy as np

samples = []

for datapath in ['./track-1', './track-2']:

    with open('./{}/driving_log.csv'.format(datapath)) as fp:
        reader = csv.reader(fp)

        adjustments = [0, 0.2, -0.2]

        for line in reader:    
            for i in range(3):
                path = './{}/IMG/{}'.format(datapath, line[i].split('/')[-1])
                measurement = float(line[3]) + adjustments[i]
                
                samples.append((path, measurement))

num_samples = len(samples)
print('Total samples: {}'.format(len(samples)))

training_samples = samples[:int(num_samples * 0.8)]
num_training_samples = len(training_samples)
print('Training samples: {}'.format(num_training_samples))

validation_samples = samples[int(num_samples * 0.8):]
num_validation_samples = len(validation_samples)
print('Validation samples: {}'.format(num_validation_samples))

In [ ]:
import matplotlib.pyplot as plt


test_image = cv2.imread(samples[0][0])
input_shape = test_image.shape
print('Input shape: {}'.format(input_shape))

plt.imshow(test_image)
plt.show()


### Load Data Set

In [ ]:
import math


epochs = 10
batch_size = 512

def batch_generator(samples):

    num_batches = math.ceil(len(samples) / batch_size)
    
    while True:

        for i in range(0, num_batches):

            start = i * batch_size
            end = start + batch_size
            batch = samples[start:end]

            batch_inputs = []
            batch_targets = []

            for path, measurement in batch:
                batch_inputs.append(cv2.imread(path))
                batch_targets.append(measurement)

            yield (np.array(batch_inputs), np.array(batch_targets))


def training_generator():
    return batch_generator(training_samples)


def validation_generator():
    return batch_generator(validation_samples)


# for out in training_generator():
#     inputs, targets = out
#     print(targets[1])
#     plt.imshow(inputs[0])
#     plt.show()

### Network Architecture

In [ ]:
import keras
from keras.models import Sequential, Model
from keras.layers.core import Dense, Activation, Flatten, Dropout, Lambda, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, Cropping2D


model = Sequential()
model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape=input_shape))
model.add(Cropping2D(cropping=((65, 25), (0, 0))))

model.add(Convolution2D(24, (5, 5), strides=(2, 2), activation='relu'))
model.add(Convolution2D(36, (5, 5), strides=(2, 2), activation='relu'))
model.add(Convolution2D(48, (5, 5), strides=(2, 2), activation='relu'))
model.add(Convolution2D(64, (3, 3), strides=(1, 1), activation='relu'))
model.add(Convolution2D(64, (3, 3), strides=(1, 1), activation='relu'))
model.add(Flatten())
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))


'''
# LeNet
model.add(Convolution2D(6, (5,5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), padding='valid'))
model.add(Convolution2D(6, (5,5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), padding='valid'))
model.add(Flatten())
model.add(Dense(120))
model.add(Dense(84))
model.add(Dense(1))
'''

model.compile(loss='mse', optimizer='adam')

# history_object = model.fit(X_train, y_train, validation_split=0.2, shuffle=True, epochs=3)

history_object = model.fit_generator(
    generator=training_generator(),
    steps_per_epoch=math.ceil(len(training_samples) / batch_size),
    validation_data=validation_generator(),
    validation_steps=math.ceil(len(validation_samples) / batch_size),
    epochs=10,
    initial_epoch=0,
    verbose=1
)

### print the keys contained in the history object
print(history_object.history.keys())

### plot the training and validation loss for each epoch
plt.plot(history_object.history['loss'])
plt.plot(history_object.history['val_loss'])
plt.title('model mean squared error loss')
plt.ylabel('mean squared error loss')
plt.xlabel('epoch')
plt.legend(['training set', 'validation set'], loc='upper right')
plt.show()


model.save('model.h5')

print('Model saved.')